In [1]:
%pylab inline
%matplotlib inline

# Global Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import os
import sys
import pickle
from pprint import pprint
from time import time
import datetime
from time import gmtime, strftime
import statsmodels.api as sm
from patsy import dmatrices

# Scikit-Learn imports
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import useful_methods

Populating the interactive namespace from numpy and matplotlib


In [2]:
def ReadEmolexDf(RESULT_FILE_NAME):
    # Read game_infos as df
    dfGameInfo = useful_methods.csv_dic_df(paths.READ_PATH_GAME_INFO + 'game_infos.csv')
    dfGameInfo = useful_methods.DropNanGames(dfGameInfo).copy().reset_index(drop=True)
    dfGameInfo.GW = [int(gw) for gw in dfGameInfo.GW]
    dfGameInfo = dfGameInfo.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)

    # Read Hash Emolex Model result
    df = useful_methods.csv_dic_df(paths.READ_PATH_RESULTS + RESULT_FILE_NAME)
    df.GW = [int(gw) for gw in df.GW]
    df = df.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)

    df.emolex_home = [np.array([float(emo.strip()) for emo in emolex_home[1:-2].split('.')]) for emolex_home in list(df.emolex_home)]
    df.emolex_away = [np.array([float(emo.strip()) for emo in emolex_away[1:-2].split('.')]) for emolex_away in list(df.emolex_away)]


    # Combine 2 dfs
    df['score_ht_away'] = [int(item) for item in dfGameInfo.score_ht_away]
    df['score_ht_home'] = [int(item) for item in dfGameInfo.score_ht_home]
    df['score_ft_away'] = [int(item) for item in dfGameInfo.score_ft_away]
    df['score_ft_home'] = [int(item) for item in dfGameInfo.score_ft_home]

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    def Labeling(goal_diff):
        if goal_diff > 0:
            return 1
        elif goal_diff < 0:
            return 0
        else:
            return 2

    df['goal_diff_ht'] = df.score_ht_home - df.score_ht_away
    df['goal_diff_ft'] = df.score_ft_home - df.score_ft_away
    df['result'] = [Labeling(item) for item in df.goal_diff_ft]


    # **********************************************
    # Add previous 4 games points sum
    dfBe4GameSum = useful_methods.csv_dic_df(paths.DATA_HOME + "EPL/info_before_4game_sum.csv")

    team_homes = list(df.home_team)
    team_aways = list(df.away_team)
    GWs = [int(gw) for gw in df.GW]

    be_4game_sum_home = []
    be_4game_sum_away = []

    for index in range(len(GWs)):
        team_home = team_homes[index]
        team_away = team_aways[index]
        gw = GWs[index]

        be_4game_sum_home.append(int(dfBe4GameSum[dfBe4GameSum.team == team_home][str(gw)]))
        be_4game_sum_away.append(int(dfBe4GameSum[dfBe4GameSum.team == team_away][str(gw)]))

    df['be_4game_sum_away'] = be_4game_sum_away
    df['be_4game_sum_home'] = be_4game_sum_home    

    return df

In [3]:
# Create df for models.
def CreateDfForModel(RESULT_FILE_NAME, ht_draw=False, ft_wld=False):
    df = ReadEmolexDf(RESULT_FILE_NAME)
        
    # only for Win or Lose
    if not ft_wld:
        df = df[df.result != 2].copy().reset_index(drop=True)
    
    # HT: Equal
    if ht_draw:
        df = df[df.goal_diff_ht == 0].copy().reset_index(drop=True)
    
    dta = pd.DataFrame()
    
    # Teams
    dta['team_home'] = df.home_team
    dta['team_away'] = df.away_team
    
    # HF scores
    dta['score_ht_home'] = df.score_ht_home
    dta['score_ht_away'] = df.score_ht_away
    
    dta['goal_diff_ht'] = df['goal_diff_ht']

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    dta['result'] = df.result
    
    # be_4game_sum
    dta['be_4game_sum_home'] = df.be_4game_sum_home / 12.0
    dta['be_4game_sum_away'] = df.be_4game_sum_away / 12.0
    
    # Emolex 8
    dta['anger_home'] = [emolex[0] / sum(emolex[:-2]) for emolex in df.emolex_home]
    dta['fear_home'] = [emolex[1] / sum(emolex[:-2]) for emolex in df.emolex_home]
    dta['disgust_home'] = [emolex[2] / sum(emolex[:-2]) for emolex in df.emolex_home]
    dta['sadness_home'] = [emolex[3] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['surprise_home'] = [emolex[4] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['trust_home'] = [emolex[5] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['joy_home'] = [emolex[6] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['anticipation_home'] = [emolex[7] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['pos_home'] = [emolex[8] / sum(emolex[-2:])  for emolex in df.emolex_home]
    dta['neg_home'] = [emolex[9] / sum(emolex[-2:])  for emolex in df.emolex_home]

    dta['anger_away'] = [emolex[0] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['fear_away'] = [emolex[1] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['disgust_away'] = [emolex[2] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['sadness_away'] = [emolex[3] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['surprise_away'] = [emolex[4] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['trust_away'] = [emolex[5] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['joy_away'] = [emolex[6] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['anticipation_away'] = [emolex[7] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['pos_away'] = [emolex[8] / sum(emolex[-2:])  for emolex in df.emolex_away]
    dta['neg_away'] = [emolex[9] / sum(emolex[-2:])  for emolex in df.emolex_away]
    
    # pos, neg from emolex8
    dta['tja_home'] = [sum(emolex[5:8]) for emolex in df.emolex_home]
    dta['afds_home'] = [sum(emolex[0:4]) for emolex in df.emolex_home]
    dta['pos_home_emo'] = dta['tja_home'] / (dta['tja_home'] + dta['afds_home'])
    dta['neg_home_emo'] = dta['afds_home'] / (dta['tja_home'] + dta['afds_home'])
    
    dta['tja_away'] = [sum(emolex[5:8]) for emolex in df.emolex_away]
    dta['afds_away'] = [sum(emolex[0:4]) for emolex in df.emolex_away]
    dta['pos_away_emo'] = dta['tja_away'] / (dta['tja_away'] + dta['afds_away'])
    dta['neg_away_emo'] = dta['afds_away'] / (dta['tja_away'] + dta['afds_away'])
    
    return dta

In [4]:
def EplEmolexTable(RESULT_FILE_NAME):
    df = CreateDfForModel(RESULT_FILE_NAME, ft_wld=True)
    df = df.drop(df.columns[2:8], axis = 1)

    teams = list(set(df.team_home))
    teams.sort()

    pos_avg = []
    neg_avg = []
    anger_avg = []
    fear_avg = []
    disgust_avg = []
    sadness_avg = []
    surprise_avg = []
    trust_avg = []
    joy_avg = []
    anticipation_avg = []
    
    pos_avg_emo = []
    neg_avg_emo = []
    
    for team in teams:
        df_home = df[(df.team_home == team)]
        df_away = df[(df.team_away == team)]
        
        # pos
        pos_home = df_home.pos_home.mean()
        pos_away = df_away.pos_away.mean()
        pos_avg.append((pos_home + pos_away) / 2)
        
        # neg
        neg_home = df_home.neg_home.mean()
        neg_away = df_away.neg_away.mean()
        neg_avg.append((neg_home + neg_away) / 2)
        
        # anger
        anger_home = df_home.anger_home.mean()
        anger_away = df_away.anger_away.mean()
        anger_avg.append((anger_home + anger_away) / 2)
        
        # fear
        fear_home = df_home.fear_home.mean()
        fear_away = df_away.fear_away.mean()
        fear_avg.append((fear_home + fear_away) / 2)
        
        # disgust
        disgust_home = df_home.disgust_home.mean()
        disgust_away = df_away.disgust_away.mean()
        disgust_avg.append((disgust_home + disgust_away) / 2)
        
        # sadness
        sadness_home = df_home.sadness_home.mean()
        sadness_away = df_away.sadness_away.mean()
        sadness_avg.append((sadness_home + sadness_away) / 2)
        
        # surprise
        surprise_home = df_home.surprise_home.mean()
        surprise_away = df_away.surprise_away.mean()
        surprise_avg.append((surprise_home + surprise_away) / 2)
        
        # trust
        trust_home = df_home.trust_home.mean()
        trust_away = df_away.trust_away.mean()
        trust_avg.append((trust_home + trust_away) / 2)
        
        # joy
        joy_home = df_home.joy_home.mean()
        joy_away = df_away.joy_away.mean()
        joy_avg.append((joy_home + joy_away) / 2)
        
        # anticipation
        anticipation_home = df_home.anticipation_home.mean()
        anticipation_away = df_away.anticipation_away.mean()
        anticipation_avg.append((anticipation_home + anticipation_away) / 2)
        
        # pos
        pos_home_emo = df_home.pos_home_emo.mean()
        pos_away_emo = df_away.pos_away_emo.mean()
        pos_avg_emo.append((pos_home_emo + pos_away_emo) / 2)
        
        # neg
        neg_home_emo = df_home.neg_home_emo.mean()
        neg_away_emo = df_away.neg_away_emo.mean()
        neg_avg_emo.append((neg_home_emo + neg_away_emo) / 2)
        
        
        
    
    # Create new df
    dfSentTable = pd.DataFrame()
    dfSentTable['team'] = teams

    dfSentTable['pos_avg'] = pos_avg
    dfSentTable['neg_avg'] = neg_avg
    dfSentTable['anger_avg'] = anger_avg
    dfSentTable['fear_avg'] = fear_avg
    dfSentTable['disgust_avg'] = disgust_avg
    dfSentTable['sadness_avg'] = sadness_avg
    dfSentTable['surprise_avg'] = surprise_avg
    dfSentTable['trust_avg'] = trust_avg
    dfSentTable['joy_avg'] = joy_avg
    dfSentTable['anticipation_avg'] = anticipation_avg
    
    dfSentTable['pos_avg_emo'] = pos_avg_emo
    dfSentTable['neg_avg_emo'] = neg_avg_emo
    
  
    return dfSentTable.sort_values(['pos_avg'], ascending=[False]).reset_index(drop=True)

## EPL Emplex Table

In [6]:
# Definings

RESULT_FILE_NAME_HT = "emolex_all_ht.csv"
RESULT_FILE_NAME_FT = "emolex_all_ft.csv"

dfEmolexTable = EplEmolexTable(RESULT_FILE_NAME_FT)
dfEmolexTable.sort_values(['pos_avg'], ascending=[False]).reset_index(drop=True)

,team,pos_avg,neg_avg,anger_avg,fear_avg,disgust_avg,sadness_avg,surprise_avg,trust_avg,joy_avg,anticipation_avg,pos_avg_emo,neg_avg_emo
0,Leicester,0.655087,0.344913,0.101322,0.086844,0.076541,0.085962,0.106358,0.202426,0.161575,0.178972,0.607860,0.392140
1,Stoke,0.628778,0.371222,0.112141,0.111658,0.066811,0.090757,0.106720,0.198186,0.152322,0.161405,0.573211,0.426789
2,Swansea,0.623561,0.376439,0.104104,0.093345,0.075620,0.088545,0.116088,0.198184,0.159560,0.164553,0.591818,0.408182
3,Bournemouth,0.614739,0.385261,0.133732,0.111281,0.065656,0.100053,0.094579,0.188663,0.149865,0.156171,0.547481,0.452519
4,Watford,0.608972,0.391028,0.116935,0.110078,0.071336,0.087394,0.115584,0.185342,0.151824,0.161506,0.565415,0.434585
5,Southampton,0.607675,0.392325,0.104916,0.096958,0.073616,0.091511,0.113325,0.176396,0.167901,0.175378,0.587936,0.412064
6,Tottenham,0.605583,0.394417,0.123491,0.090631,0.067782,0.102222,0.100482,0.178940,0.176015,0.160437,0.572969,0.427031
7,Arsenal,0.599730,0.400270,0.111489,0.101805,0.080686,0.099386,0.099285,0.199849,0.150461,0.157039,0.564569,0.435431
8,WestHam,0.592103,0.407897,0.123260,0.117812,0.074199,0.101504,0.097762,0.186063,0.149956,0.149445,0.538402,0.461598
9,Norwich,0.579195,0.420805,0.109560,0.107553,0.081789,0.106055,0.107364,0.181100,0.146129,0.160449,0.548588,0.451412


## EPL Score Table
Exclude GW: [1, 2, 3, 4, 13]

In [ ]:
dfScore = useful_methods.ReadEplScoreTable()

# drop weeks
dfScore = dfScore.drop(dfScore.columns[[1, 2, 3, 4, 5, 13]], axis=1)

# sum all scores
dfScore['pts'] = dfScore.sum(axis=1)

# sort by overall points
dfScore = dfScore.sort_values(['pts'], ascending=False).reset_index(drop=True)
dfScore